В этой работе мы реализуем переводчик с русского языка на язык Старшей Речи и наоборот. Тоже самое с английским.
Реализуем следующими способами:
- Прямым алгоритмом
- Моделями:
1. Deepseek
2. Llama
3. Alpaca

На данный момент реализован только пункт 1 с русского на Старшую Речь.

In [4]:
import pandas as pd

import requests
import re
import string

import pymorphy2
lemmer = pymorphy2.MorphAnalyzer()

In [5]:
df = pd.read_csv('../lr3/witcher_words.csv')

In [63]:
def translate_russian_to_elvish(text: str) -> str:
    """
    Переводит русский текст на язык Старшей Речи.
    Слова, перевод которым подобрать не получилось, латинизируются.
    Присутствует обработка некоторых устойчивых выражений из словаря.
    """

    # Унифицируем е/ё:
    text = text.replace('ё', 'е')

    # Результирующие подстроки для склеивания в конце.
    result_subs = []

    # Отделяем слова от пунктуационных знаков:
    subs = re.split(r'(?<=\.|!|\?|,|;|:|\s|\n|\(|\))|(?=\.|!|\?|,|;|:|\s|\n|\(|\))', text)

    next_i = 0
    for i, sub in enumerate(subs):
        if i < next_i:
            continue

        next_i += 1
        if sub in string.punctuation + ' \n':
            result_subs.append(sub)
            continue

        word = sub
        is_title = word.replace('\'', '').istitle()
        word = word.lower()
        word = _word_to_lemm(word)

        # Текущее слово может быть началом какого-то устойчивого выражения, проверим это.
        it_is_real_phrase = False
        phrases = _find_phrases_that_starts_with_russian_word(word)
        if phrases and i != len(subs) - 1:
            # Будем прибавлять сюда последующие пробелы и слова, пока не встретим что-то лишнее либо не убедимся
            # в том, что это выражение - действительная устойчивая фраза из словаря.
            summ_word = word

            # Количество итераций для пропуска во внешнем цикле (через `next_i`)
            count_to_pass = 0

            for sub in subs[i + 1:]:
                if sub in '.!?':
                    break

                count_to_pass += 1
                if sub not in string.punctuation + ' ':
                    sub = _word_to_lemm(sub.lower())
                elif sub == ',':
                    # Опускаем запятые между словами потенциальной фразы.
                    sub = ''

                summ_word += sub

                need_continue = False
                if summ_word in phrases:
                    it_is_real_phrase = True
                    break

                for phrase in phrases:
                    if summ_word in phrase:
                        need_continue = True
                        break

                if not need_continue:
                    break

            if it_is_real_phrase:
                word = summ_word
                next_i += count_to_pass
                translation = _try_to_find_russian_word_translation(word)

        is_bad = False
        if not it_is_real_phrase:
            # Стандартная обработка одиночного слова с подбором синонимов через сайт.
            translation = None
            try:
                translation = _try_to_find_russian_word_translation(word)
            except ValueError:
                for synonym in _synonyms_of_word(word):
                    try:
                        translation = _try_to_find_russian_word_translation(synonym)
                    except ValueError:
                        continue

            if translation is None:
                translation = _transliterate_russian_to_latin(word)
                is_bad = True

        if is_title:
            translation = translation.capitalize()

        if is_bad:
            # Добавим подчеркивание "неудачного" слова:
            translation = f'\033[4m{translation}\033[0m'

        result_subs.append(translation)

    return ''.join(result_subs)


def _word_to_lemm(word: str) -> str:
    """Переводит слово/словосочетание в начальную форму."""
    parts = []
    for part in word.split():
        part = lemmer.parse(part)[0].normal_form.replace('ё', 'е')
        parts.append(part)

    return ' '.join(parts)


def _try_to_find_russian_word_translation(word: str) -> str:
    """Ищет перевод русского слова/словосочетания в датасете."""
    result = df.loc[df['translation'] == word, 'text']
    if not result.empty:
        return result.values[0]
    else:
        raise ValueError


def _find_phrases_that_starts_with_russian_word(word: str) -> list[str]:
    return df.loc[df['translation'].str.startswith(word) & df['translation'].str.contains(' '), 'translation'].tolist()


def _synonyms_of_word(word: str) -> str:
    """Выдаёт всевозможные синонимы заданного слова. Использует сетевой запрос."""
    html = requests.get(f'https://text.ru/synonym/{word}').text
    
    match = re.search(r'<meta name=\"description\" content=\"Синонимы к слову [^—]*:([^\"]+)\" />', html)
    if match is None:
        return []

    text = match.group(1)
    synonyms = [synonym.strip() for synonym in text.strip().split('—')]
    synonyms.remove('')

    return synonyms


def _transliterate_russian_to_latin(russian_word: str) -> str:
    """Функция для латинизации русского слова."""
    translit_dict = {
        'а': 'a', 
        'б': 'b', 
        'в': 'v', 
        'г': 'g', 
        'д': 'd', 
        'е': 'e', 
        'ё': 'yo',
        'ж': 'zh', 
        'з': 'z', 
        'и': 'i', 
        'й': 'y', 
        'к': 'k', 
        'л': 'l', 
        'м': 'm',
        'н': 'n', 
        'о': 'o', 
        'п': 'p', 
        'р': 'r', 
        'с': 's', 
        'т': 't', 
        'у': 'u',
        'ф': 'f', 
        'х': 'kh', 
        'ц': 'ts', 
        'ч': 'ch', 
        'ш': 'sh', 
        'щ': 'shch', 
        'ъ': '',
        'ы': 'y', 
        'ь': '', 
        'э': 'e', 
        'ю': 'yu', 
        'я': 'ya',
    }

    latin_word = ''.join(translit_dict.get(symbol, symbol) for symbol in russian_word.lower())
    return latin_word

Попробуем посмотреть, как работает.

In [64]:
print(translate_russian_to_elvish('Здравствуй, Геральт Белый Волк! Моё имя - Аваллак\'х! Помоги мне найти ласточку!'))

Cead, Geralt Gwynbleidd! Me aymm - Avallak'kha! Pomoch aé nayti zireael!


In [65]:
print(translate_russian_to_elvish('Прощай, дочь. Мы еще встретимся.'))

Va faill, luned. Sinn vort vstretitsya.


In [66]:
print(translate_russian_to_elvish('Когда обед? Я хочу есть.'))

Cathain obed? Aé wett ithte.


In [67]:
print(translate_russian_to_elvish('Почему этот курс такой сложный? Я не знаю.'))

Que etot rider takoy slozhnyy? Aé neen ifit.


In [68]:
print(translate_russian_to_elvish('Крепость Старого Моря прекрасна летом. Он здесь как картина.'))

Caer a'muirehen elaine eate. Ei anseo conas eie.


In [69]:
print(translate_russian_to_elvish('Привет, я в порядке, спасибо.'))

Cead, esseath ghnath, meas.


In [70]:
print(translate_russian_to_elvish('Что ты хочешь?'))

Que te eras'wett?


In [71]:
print(translate_russian_to_elvish('Здарова! Привет всем!'))

Zdarov! Ceadmil!


In [72]:
print(translate_russian_to_elvish('Не забывай, Цири. Всегда, когда что-то кончается, что-то начинается.'))

Neen zabyvat, Tsiri. Evellienn, cathain va'esse deireadh aep eigean va'esse eighe faidh'ar.


In [73]:
print(translate_russian_to_elvish('Это в настоящее время нельзя делать.'))

А faoi lathair ayd tuve.


In [74]:
print(translate_russian_to_elvish('Ты что! Следи за огнем! Костер почти прогорел.'))

Esseath que! Aereeh't do aenye! An wenn beagnach muchfar.


In [75]:
print(translate_russian_to_elvish("""
Нужно сказать, что Сапковский не из воздуха взял основу для своего языка, а зиждился на реальных естественных языках.
К примеру, эльфийские слова blaec и tief напоминают английские black и thief соответственно.
Есть и полностью заимствованные слова (uaigh из ирландского, что значит могила) и просто образованные путём изменения
изначальных слов (dearme (спать) практически одинакого с dream). Есть и полностью придуманные слова,
или изменённые настолько, что нельзя понять, что служило для них основой.
"""))


Nuzhno skazat, que Sapkovskiy neen al eire vzyat peor't а ar yazyk, que zizhditsya yn dearbh dearbh yazyk.
А primer, elfiyskiy speath blaec estoll tief napominat angliyskiy black estoll thief sootvetstvenno.
Ithte estoll evellienn zaimstvovat speath (uaigh al irlandskiy, que ifit uaigh) estoll prosto obrazovat thar feabh'as
iznachalnyy speath (dearme (dearme) le odinakiy aep dream). Ithte estoll evellienn pridumat speath,
aevon izmenit nastolko, que ayd ponyat, que sluzhit а iad peor't.

